**Based on Zaccone: Python Parallel Programming Cookbook** 

### Thread synchronization with a condition

A condition identifies a change of state in the application. This is a synchronization 
mechanism where a thread waits for a specific condition and another thread notifies that this 
condition has taken place. Once the condition takes place, the thread acquires the lock to get 
exclusive access to the shared resource.

A good way to illustrate this mechanism is by looking again at a producer/consumer problem:

* The class **producer** writes to a buffer as long as it is not full

* The class **producer** will notify the consumer that the buffer is not empty, 

* the class **consumer** takes the data from the buffer (eliminating them from the latter), as long as the buffer is not empty. 

* while the **consumer** will report to the producer that the buffer is not full.

In [1]:
# import Condition!
from threading import Thread, Condition
import time

items = []
condition = Condition()


In [2]:
class consumer(Thread):
    def __init__(self):
        Thread.__init__(self)

    def consume(self):
        global condition
        global items
        
        # The class consumer acquires the shared resource that is modeled through the list items[]:
        condition.acquire()
        
        #If the length of the list is equal to 0, the consumer is placed in a waiting state:
        if len(items) == 0:
            condition.wait()
            print("Consumer notify : no item to consume")
            
        #Otherwise, it makes a pop operation from the items list:
        items.pop()
        print("Consumer notify : consumed 1 item")
        print("Consumer notify : items to consume are "+ str(len(items)))
        
        #The consumer's state is notified to the producer and the shared resource is released:
        condition.notify()
        condition.release()

    def run(self):
        # run 10 iterations
        for i in range(0,10):
            time.sleep(3)
            self.consume()


In [3]:
class producer(Thread):
    def __init__(self):
        Thread.__init__(self)

    def produce(self):
        global condition
        global items

        # The class producer acquires the shared resource and then it verifies that the list is completely 
        # full (in our example, we place the maximum number of items, 10, that can be contained in 
        # the items list). 
        # If the list is full, then the producer is placed in the wait state until the list is  consumed:
        
        condition.acquire()
        if len(items) == 10:
            condition.wait()
            print("Producer notify : items producted are " + str(len(items)))
            print("Producer notify : stop the production!!")
        items.append(1)
        print("Producer notify : total items producted " + str(len(items)))
        condition.notify()
        condition.release()

    def run(self):
        # run 10 iterations
        for i in range(0,20):
            time.sleep(1)
            self.produce()



In [4]:
producer = producer()
consumer = consumer()
producer.start()
consumer.start()
producer.join()
consumer.join()


Producer notify : total items producted 1
Producer notify : total items producted 2
Producer notify : total items producted 3
Consumer notify : consumed 1 item
Consumer notify : items to consume are 2
Producer notify : total items producted 3
Producer notify : total items producted 4
Consumer notify : consumed 1 item
Consumer notify : items to consume are 3
Producer notify : total items producted 4
Producer notify : total items producted 5
Producer notify : total items producted 6
Consumer notify : consumed 1 item
Consumer notify : items to consume are 5
Producer notify : total items producted 6
Producer notify : total items producted 7
Producer notify : total items producted 8
Consumer notify : consumed 1 item
Consumer notify : items to consume are 7
Producer notify : total items producted 8
Producer notify : total items producted 9
Producer notify : total items producted 10
Consumer notify : consumed 1 item
Consumer notify : items to consume are 9
Producer notify : total items produc